# Shoreline-grayscale

Firstly let's try to connect to Earth Engine with the python package

In [1]:
import ee

def authenticate_and_initialize():
    try:
        # Authenticate the Earth Engine session.
        ee.Authenticate()
        # Initialize the Earth Engine module.
        ee.Initialize()
        print("Authentication successful!")
    except Exception as e:
        print(f"Authentication failed: {e}")

if __name__ == "__main__":
    authenticate_and_initialize()

Authentication successful!


# load images from Earth Engine 

Users can add their polygon if they want via user input and advanced user could directly modify the JSON

In [3]:
import json

# Ask the user if they want to add a new site
add_new_site = input("Do you want to add a new site? (yes/no): ").strip().lower()

if add_new_site == 'yes':
    try:
        # Ask the user to enter the new site's information
        sitename = input("Enter the site name (sitename): ")
        country = input("Enter the country of the site (country): ")

        print("Enter the polygon coordinates. Type 'done' when you are finished.")
        polygon = []
        while True:
            coords = input("Enter a coordinate in the form 'latitude,longitude': ")
            if coords.lower() == 'done':
                break
            try:
                lat, lon = map(float, coords.split(','))
                polygon.append([lat, lon])
            except ValueError:
                print("Invalid format. Make sure to enter the coordinates in the form 'latitude,longitude'.")

        # Ensure the polygon is closed by adding the first point to the end
        if polygon and polygon[0] != polygon[-1]:
            polygon.append(polygon[0])

        # Load the existing data from the JSON file
        try:
            with open('sites_data.json', 'r') as json_file:
                data_loaded = json.load(json_file)
        except FileNotFoundError:
            print("The file 'sites_data.json' does not exist. Creating a new file.")
            data_loaded = {"sites": []}
        except json.JSONDecodeError:
            print("Error decoding JSON from the file. Please check the file format.")
            data_loaded = {"sites": []}

        # Add the new site data to the loaded dictionary
        new_site = {
            "sitename": sitename,
            "country": country,
            "polygon": [polygon]
        }
        data_loaded["sites"].append(new_site)

        # Write the updated data back to the JSON file
        try:
            with open('sites_data.json', 'w') as json_file:
                json.dump(data_loaded, json_file, indent=4)
            print("Data loaded correctly.")
        except IOError as e:
            print(f"An error occurred while writing to the file: {e}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
    print("No changes made.")


No changes made.


In [ ]:
#Import coastsat functions 

In [2]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects

# Make a lots of datas

In [6]:
import json

# Define date range
dates = ['1984-01-01', '2025-01-01']

fps = 4

# Define satellite missions
sat_list = ['L5', 'L7', 'L8', 'L9']

# Choose Landsat collection
collection = 'C02'

# Directory where the data will be stored
filepath = os.path.join(os.getcwd(), 'data')

settings = {
    # General parameters:
    'cloud_thresh': 0.5,        # Threshold on maximum cloud cover
    'dist_clouds': 300,         # Distance around clouds where shoreline can't be mapped
    'output_epsg': 28356,       # EPSG code of spatial reference system desired for the output
    # Quality control:
    'check_detection': True,    # If True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # If True, allows user to adjust the position of each shoreline by changing the threshold
    'save_figure': True,        # If True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] Shoreline detection parameters:
    'min_beach_area': 1000,     # Minimum area (in metres^2) for an object to be labelled as a beach
    'min_length_sl': 500,       # Minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # Switch this parameter to True if sand pixels are masked (in black) on many images
    'sand_color': 'default',    # 'default', 'latest', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    'pan_off': False,           # True to switch pansharpening off for Landsat 7/8/9 imagery
    's2cloudless_prob': 60,     # Probability threshold to identify cloudy pixels in the s2cloudless mask
}

# Load the JSON file
try:
    with open('json/sites_data.json', 'r') as json_file:
        data_loaded = json.load(json_file)
        print("JSON data loaded successfully.")
except FileNotFoundError:
    print("The file 'sites_data.json' does not exist.")
    data_loaded = {"sites": []}
except json.JSONDecodeError:
    print("Error decoding JSON from the file. Please check the file format.")
    data_loaded = {"sites": []}
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    data_loaded = {"sites": []}

# Process each site and add inputs to each site dictionary
for site in data_loaded["sites"]:
    try:
        sitename = site['sitename']
        country = site.get('country', 'Unknown')  # Default to 'Unknown' if not present
        polygon = site['polygon']

        # Convert the polygon to the smallest rectangle (sides parallel to coordinate axes)
        polygon = SDS_tools.smallest_rectangle(polygon)
        print(f"Processed polygon for site: {sitename}")

        # Put all the inputs into a dictionary
        inputs = {
            'polygon': polygon,
            'dates': dates,
            'sat_list': sat_list,
            'sitename': sitename,
            'filepath': filepath,
            'landsat_collection': collection
        }

        # Add the inputs dictionary to the site dictionary
        site['inputs'] = inputs
        print(f"Inputs for site {sitename}: {inputs}")

    except KeyError as e:
        print(f"Key error: {e} for site: {site}")
    except Exception as e:
        print(f"An unexpected error occurred while processing site: {sitename}. Error: {e}")

# Save the updated JSON file with inputs added
try:
    with open('json/sites_data_updated.json', 'w') as json_file:
        json.dump(data_loaded, json_file, indent=4)
    print("Updated JSON data saved successfully.")
except Exception as e:
    print(f"An error occurred while saving the updated JSON data: {e}")

# Use the updated JSON file for further processing
for site in data_loaded["sites"]:
    try:
        sitename = site['sitename']
        inputs = site['inputs']

        # Debug: print inputs to trace errors
        print(f"Debug: Inputs for site {sitename}: {inputs}")

        # Merge settings with inputs for this site
        site_settings = settings.copy()
        site_settings.update(inputs)  # Directly update settings with inputs
        site_settings['inputs'] = inputs  # Also add inputs as a sub-dictionary in settings

        # Check if shoreline animation already exists
        fn_animation_shorelines = os.path.join(inputs['filepath'], inputs['sitename'], f'{inputs["sitename"]}_animation_shorelines.mp4')
        fn_animation_rgb = os.path.join(inputs['filepath'], inputs['sitename'], f'{inputs["sitename"]}_animation_RGB.mp4')
        
        if os.path.exists(fn_animation_shorelines) and os.path.exists(fn_animation_rgb):
            print(f"Both animations already exist for site: {sitename}, skipping shoreline mapping and animation creation.")
            continue

        # Before downloading the images, check how many images are available for your inputs
        SDS_download.check_images_available(site_settings)
        print(f"Checked image availability for site: {sitename}")
        metadata = SDS_download.retrieve_images(site_settings)
        metadata2 = SDS_download.get_metadata(site_settings)

        # Check if images already exist
        image_path = os.path.join(inputs['filepath'], inputs['sitename'], 'jpg_files', 'preprocessed')
        if not os.path.exists(image_path):
            os.makedirs(image_path)
        
        if not any(os.scandir(image_path)):  # If directory is empty
            # Save the images
            SDS_preprocess.save_jpg(metadata, site_settings, use_matplotlib=True)
            print(f"Images saved for site: {sitename}")
        else:
            print(f"Images already exist for site: {sitename}, skipping save_jpg.")

        # Process shoreline extraction and reference shoreline if shorelines animation does not exist
        if not os.path.exists(fn_animation_shorelines):
            print("Getting reference shoreline...")
            settings['reference_shoreline'] = SDS_preprocess.get_reference_sl(metadata, site_settings)
            settings['max_dist_ref'] = 100  # max distance (in meters) allowed from the reference shoreline
            print("Extracting shorelines...")
            
            output = SDS_shoreline.extract_shorelines(metadata, site_settings)
            print("Removing duplicates...")
            output = SDS_tools.remove_duplicates(output)  # removes duplicates (images taken on the same date by the same satellite)
            output = SDS_tools.remove_inaccurate_georef(output, 10)  # remove inaccurate georeferencing (set threshold to 10 m)

            # Create MP4 timelapse animation of shorelines
            fp_images = os.path.join(inputs['filepath'], inputs['sitename'], 'jpg_files', 'detection')
            fps = 4  # frames per second in animation
            print("Creating animation MP4 of shorelines...")
            SDS_tools.make_animation_mp4(fp_images, fps, fn_animation_shorelines)
            print(f"Shoreline animation created for site: {sitename}")

            # Plot and save the shoreline data
            fig = plt.figure(figsize=[15, 8])
            plt.axis('equal')
            plt.xlabel('Eastings')
            plt.ylabel('Northings')
            plt.grid(linestyle=':', color='0.5')
            for i in range(len(output['shorelines'])):
                sl = output['shorelines'][i]
                date = output['dates'][i]
                plt.plot(sl[:, 0], sl[:, 1], '.', label=date.strftime('%d-%m-%Y'))
            plt.legend()
            plt.title(f'Shorelines for site: {sitename}')
            plt.savefig(os.path.join(inputs['filepath'], inputs['sitename'], 'shoreline_all_time.jpg'))
            plt.close()
            print(f"Shoreline plot saved for site: {sitename}")

        # Create MP4 timelapse animation of RGB images if it does not exist
        if not os.path.exists(fn_animation_rgb):
            fp_images_rgb = os.path.join(inputs['filepath'], inputs['sitename'], 'jpg_files', 'preprocessed')
            SDS_tools.make_animation_mp4(fp_images_rgb, fps, fn_animation_rgb)
            print(f"RGB animation created for site: {sitename}")
        else:
            print(f"RGB animation already exists for site: {sitename}")

        # Shoreline analysis: Compute time-series of cross-shore distance along user-defined shore-normal transects
        print("Starting shoreline analysis...")
        filepath = os.path.join(inputs['filepath'], sitename)
        try:
            with open(os.path.join(filepath, sitename + '_output.pkl'), 'rb') as f:
                output = pickle.load(f)
                print("Shoreline output loaded successfully.")
            # Remove duplicates and inaccurate georeferencing
            output = SDS_tools.remove_duplicates(output)
            output = SDS_tools.remove_inaccurate_georef(output, 10)
            print("Shoreline analysis completed.")
        except FileNotFoundError:
            print(f"Output file for site {sitename} not found. Skipping analysis.")
        except pickle.PickleError:
            print(f"Error loading pickle file for site {sitename}.")
        except Exception as e:
            print(f"An unexpected error occurred during shoreline analysis for site {sitename}: {e}")

    except KeyError as e:
        print(f"Key error: {e} for site: {site}")
    except Exception as e:
        print(f"An unexpected error occurred while processing site: {sitename}. Error: {e}")

print("Processing completed.")

JSON data loaded successfully.
Processed polygon for site: NARRA
Inputs for site NARRA: {'polygon': [[[151.2957545, -33.7390216], [151.312234, -33.7390216], [151.312234, -33.7012561], [151.2957545, -33.7012561], [151.2957545, -33.7390216]]], 'dates': ['1984-01-01', '2025-01-01'], 'sat_list': ['L5', 'L7', 'L8', 'L9'], 'sitename': 'NARRA', 'filepath': '/home/athena/Document/Shoreline-Grayscale/data', 'landsat_collection': 'C02'}
Processed polygon for site: Dunkerque
Inputs for site Dunkerque: {'polygon': [[[2.22, 51.03], [2.29, 51.03], [2.29, 51.06], [2.22, 51.06], [2.22, 51.03]]], 'dates': ['1984-01-01', '2025-01-01'], 'sat_list': ['L5', 'L7', 'L8', 'L9'], 'sitename': 'Dunkerque', 'filepath': '/home/athena/Document/Shoreline-Grayscale/data', 'landsat_collection': 'C02'}
Processed polygon for site: DUNKERQUEPORT
Inputs for site DUNKERQUEPORT: {'polygon': [[[2.11, 51.01], [2.21, 51.01], [2.21, 51.04], [2.11, 51.04], [2.11, 51.01]]], 'dates': ['1984-01-01', '2025-01-01'], 'sat_list': ['L5'

MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


Reference shoreline has been saved in /home/athena/Document/Shoreline-Grayscale/data/Tobermory
Extracting shorelines...
Mapping shorelines:
L5:   0%Automatic mode activated
L5:   27%Skipped image '1991-10-03-10-51-52' due to high cloud cover: 0.58
L5:   45%Skipped image '1996-05-09-10-39-02' due to high cloud cover: 0.58
L5:   71%Skipped image '2003-07-16-11-04-41' due to high cloud cover: 0.75
L5:   100%
L7:   28%Skipped image '2007-04-30-11-18-44' due to high cloud cover: 0.76
L7:   31%Skipped image '2011-08-15-11-21-17' due to high cloud cover: 0.74
L7:   31%Skipped image '2011-08-15-11-21-41' due to high cloud cover: 0.74
L7:   33%Skipped image '2012-06-14-11-22-17' due to high cloud cover: 0.58
L7:   34%Skipped image '2012-09-11-11-17-30' due to high cloud cover: 0.69
L7:   35%Skipped image '2012-10-13-11-17-48' due to high cloud cover: 0.51
L7:   36%Skipped image '2013-04-14-11-23-53' due to high cloud cover: 0.68
L7:   39%Skipped image '2013-09-14-11-17-44' due to high cloud cov

[rawvideo @ 0x5e22700] Stream #0: not enough frames to estimate rate; consider increasing probesize


Animation has been generated (using 4 frames per second) and saved at /home/athena/Document/Shoreline-Grayscale/data/Tobermory/Tobermory_animation_shorelines.mp4
Shoreline animation created for site: Tobermory
Shoreline plot saved for site: Tobermory


[rawvideo @ 0x6d2f700] Stream #0: not enough frames to estimate rate; consider increasing probesize


Animation has been generated (using 4 frames per second) and saved at /home/athena/Document/Shoreline-Grayscale/data/Tobermory/Tobermory_animation_RGB.mp4
RGB animation created for site: Tobermory
Starting shoreline analysis...
Shoreline output loaded successfully.
466 duplicates
59 bad georef
Shoreline analysis completed.
Debug: Inputs for site Millport: {'polygon': [[[-4.929817, 55.748885], [-4.9131, 55.748885], [-4.9131, 55.75784], [-4.929817, 55.75784], [-4.929817, 55.748885]]], 'dates': ['1984-01-01', '2025-01-01'], 'sat_list': ['L5', 'L7', 'L8', 'L9'], 'sitename': 'Millport', 'filepath': '/home/athena/Document/Shoreline-Grayscale/data', 'landsat_collection': 'C02'}
Number of images available between 1984-01-01 and 2025-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     L5: 483 images
     L7: 388 images
     L8: 348 images
     L9: 90 images
  Total to download: 1309 images
- In Landsat Tier 2 (not suitable for time-series analysis):
     L5: 270 images
     L7: 52 images
   

# Transects implementation

In [3]:
import os
import json
import numpy as np
import pickle
import matplotlib
matplotlib.use('Agg')  # Use Agg backend for headless environments
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd
from coastsat import SDS_transects

def load_json(filepath):
    try:
        with open(filepath, 'r') as json_file:
            data = json.load(json_file)
            print("JSON data loaded successfully.")
            return data
    except FileNotFoundError:
        print(f"The file '{filepath}' does not exist.")
        return {"sites": []}
    except json.JSONDecodeError:
        print("Error decoding JSON from the file. Please check the file format.")
        return {"sites": []}
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return {"sites": []}

def load_pickle(filepath):
    try:
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
            print("Pickle file loaded successfully.")
            return data
    except FileNotFoundError:
        print(f"File '{filepath}' not found.")
        return None
    except pickle.PickleError:
        print(f"Error loading pickle file '{filepath}'.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while loading pickle file '{filepath}': {e}")
        return None

def save_pickle(data, filepath):
    try:
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)
            print(f"Data saved to '{filepath}' successfully.")
    except Exception as e:
        print(f"An unexpected error occurred while saving to '{filepath}': {e}")

def save_csv(data, filepath):
    try:
        data.to_csv(filepath, sep=',')
        print(f"CSV saved to '{filepath}' successfully.")
    except Exception as e:
        print(f"An unexpected error occurred while saving CSV to '{filepath}': {e}")

def compute_intersection_QC(output, transects, settings):
    cross_dist = dict([])
    shorelines = output['shorelines']
    along_dist = settings['along_dist']

    for key in transects.keys():
        std_intersect = np.zeros(len(shorelines))
        med_intersect = np.zeros(len(shorelines))
        max_intersect = np.zeros(len(shorelines))
        min_intersect = np.zeros(len(shorelines))
        n_intersect = np.zeros(len(shorelines))
        
        for i in range(len(shorelines)):
            sl = shorelines[i]
            X0 = transects[key][0,0]
            Y0 = transects[key][0,1]
            temp = np.array(transects[key][-1,:]) - np.array(transects[key][0,:])
            phi = np.arctan2(temp[1], temp[0])
            Mrot = np.array([[np.cos(phi), np.sin(phi)],[-np.sin(phi), np.cos(phi)]])
            p1 = np.array([X0,Y0])
            p2 = transects[key][-1,:]
            d_line = np.abs(np.cross(p2-p1,sl-p1)/np.linalg.norm(p2-p1))
            d_origin = np.array([np.linalg.norm(sl[k,:] - p1) for k in range(len(sl))])
            idx_dist = np.logical_and(d_line <= along_dist, d_origin <= 1000)
            idx_close = np.where(idx_dist)[0]

            if len(idx_close) == 0:
                std_intersect[i] = np.nan
                med_intersect[i] = np.nan
                max_intersect[i] = np.nan
                min_intersect[i] = np.nan
                n_intersect[i] = np.nan
            else:
                xy_close = np.array([sl[idx_close,0],sl[idx_close,1]]) - np.tile(np.array([[X0],[Y0]]), (1,len(sl[idx_close])))
                xy_rot = np.matmul(Mrot, xy_close)
                xy_rot[0, xy_rot[0,:] < settings['min_chainage']] = np.nan

                if not np.isnan(xy_rot[0,:]).all():
                    std_intersect[i] = np.nanstd(xy_rot[0,:])
                    med_intersect[i] = np.nanmedian(xy_rot[0,:])
                    max_intersect[i] = np.nanmax(xy_rot[0,:])
                    min_intersect[i] = np.nanmin(xy_rot[0,:])
                    n_intersect[i] = np.sum(~np.isnan(xy_rot[0, :]))
                else:
                    std_intersect[i] = np.nan
                    med_intersect[i] = np.nan
                    max_intersect[i] = np.nan
                    min_intersect[i] = np.nan
                    n_intersect[i] = np.nan

        condition1 = std_intersect <= settings['max_std']
        condition2 = (max_intersect - min_intersect) <= settings['max_range']
        condition3 = n_intersect >= settings['min_points']
        idx_good = np.logical_and(np.logical_and(condition1, condition2), condition3)

        if settings['multiple_inter'] == 'auto':
            prc_over = np.sum(std_intersect > settings['max_std'])/len(std_intersect)
            if prc_over > settings['auto_prc']:
                med_intersect[~idx_good] = max_intersect[~idx_good]
                med_intersect[~condition3] = np.nan
            else:
                med_intersect[~idx_good] = np.nan
        elif settings['multiple_inter'] == 'max':
            med_intersect[~idx_good] = max_intersect[~idx_good]
            med_intersect[~condition3] = np.nan
            prc_over = 0
        elif settings['multiple_inter'] == 'nan':
            med_intersect[~idx_good] = np.nan
            prc_over = 0
        else:
            raise Exception('the multiple_inter parameter can only be: nan, max or auto')

        cross_dist[key] = med_intersect

    return cross_dist

# Load the updated JSON file with inputs added
data_loaded = load_json('json/sites_data_updated.json')

# Process each site to define transects and generate time-series
for site in data_loaded["sites"]:
    try:
        sitename = site['sitename']
        inputs = site['inputs']
        print(f"Debug: Inputs for site {sitename}: {inputs}")

        filepath = os.path.join(inputs['filepath'], sitename)
        output = load_pickle(os.path.join(filepath, sitename + '_output.pkl'))
        if output is None:
            print(f"Skipping transect processing for site {sitename} due to missing output file.")
            continue

        transects_file = os.path.join(filepath, sitename + '_transects.pkl')
        if os.path.exists(transects_file):
            transects = load_pickle(transects_file)
            print(f"Transects file already exists for site: {sitename}. Skipping transect drawing.")
        else:
            settings = {
                'cloud_thresh': 0.5,
                'dist_clouds': 300,
                'output_epsg': 28356,
                'check_detection': True,
                'adjust_detection': False,
                'save_figure': True,
                'min_beach_area': 1000,
                'min_length_sl': 500,
                'cloud_mask_issue': False,
                'sand_color': 'default',
                'pan_off': False,
                's2cloudless_prob': 60,
                'inputs': inputs,
            }

            print(f"Drawing transects interactively for site: {sitename}")
            %matplotlib qt
            transects = SDS_transects.draw_transects(output, settings)
            save_pickle(transects, transects_file)

        settings_transects = {'along_dist': 25}
        cross_distance = SDS_transects.compute_intersection(output, transects, settings_transects)

        settings_transects_qc = {
            'along_dist': 25,
            'min_points': 3,
            'max_std': 15,
            'max_range': 30,
            'min_chainage': -100,
            'multiple_inter': 'auto',
            'auto_prc': 0.1,
        }
        cross_distance_qc = compute_intersection_QC(output, transects, settings_transects_qc)

        fig = plt.figure(figsize=[15, 8], tight_layout=True)
        gs = gridspec.GridSpec(len(cross_distance_qc), 1)
        gs.update(left=0.05, right=0.95, bottom=0.05, top=0.95, hspace=0.05)
        for i, key in enumerate(cross_distance_qc.keys()):
            if np.all(np.isnan(cross_distance_qc[key])):
                continue
            ax = fig.add_subplot(gs[i, 0])
            ax.grid(linestyle=':', color='0.5')
            ax.plot(output['dates'], cross_distance_qc[key], '-o', ms=6, mfc='w')
            ax.set_ylabel('distance [m]', fontsize=12)
            ax.text(0.5, 0.95, key, bbox=dict(boxstyle="square", ec='k', fc='w'), ha='center',
                    va='top', transform=ax.transAxes, fontsize=14)
        plt.savefig(os.path.join(filepath, 'transects_time_series.jpg'))
        plt.close()
        print(f"Time-series plotted for site: {sitename}")

        out_dict = dict([])
        out_dict['dates'] = output['dates']
        for key in transects.keys():
            out_dict[key] = cross_distance_qc[key]
        df = pd.DataFrame(out_dict)
        fn = os.path.join(filepath, 'transect_time_series.csv')
        save_csv(df, fn)

    except KeyError as e:
        print(f"Key error: {e} for site: {site}")
    except Exception as e:
        print(f"An unexpected error occurred while processing transects for site: {sitename}. Error: {e}")

print("Transect processing completed.")


JSON data loaded successfully.
Debug: Inputs for site NARRA: {'polygon': [[[151.2957545, -33.7390216], [151.312234, -33.7390216], [151.312234, -33.7012561], [151.2957545, -33.7012561], [151.2957545, -33.7390216]]], 'dates': ['1984-01-01', '2025-01-01'], 'sat_list': ['L5', 'L7', 'L8', 'L9'], 'sitename': 'NARRA', 'filepath': '/home/athena/Document/Shoreline-Grayscale/data', 'landsat_collection': 'C02'}
Pickle file loaded successfully.
Pickle file loaded successfully.
Transects file already exists for site: NARRA. Skipping transect drawing.
Time-series plotted for site: NARRA
CSV saved to '/home/athena/Document/Shoreline-Grayscale/data/NARRA/transect_time_series.csv' successfully.
Debug: Inputs for site Dunkerque: {'polygon': [[[2.22, 51.03], [2.29, 51.03], [2.29, 51.06], [2.22, 51.06], [2.22, 51.03]]], 'dates': ['1984-01-01', '2025-01-01'], 'sat_list': ['L5', 'L7', 'L8', 'L9'], 'sitename': 'Dunkerque', 'filepath': '/home/athena/Document/Shoreline-Grayscale/data', 'landsat_collection': 'C

MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


An unexpected error occurred while processing transects for site: Dunkerque. Error: list index out of range
Debug: Inputs for site DUNKERQUEPORT: {'polygon': [[[2.11, 51.01], [2.21, 51.01], [2.21, 51.04], [2.11, 51.04], [2.11, 51.01]]], 'dates': ['1984-01-01', '2025-01-01'], 'sat_list': ['L5', 'L7', 'L8', 'L9'], 'sitename': 'DUNKERQUEPORT', 'filepath': '/home/athena/Document/Shoreline-Grayscale/data', 'landsat_collection': 'C02'}
Pickle file loaded successfully.
Pickle file loaded successfully.
Transects file already exists for site: DUNKERQUEPORT. Skipping transect drawing.
Time-series plotted for site: DUNKERQUEPORT
CSV saved to '/home/athena/Document/Shoreline-Grayscale/data/DUNKERQUEPORT/transect_time_series.csv' successfully.
Debug: Inputs for site HEMMESDEMARCK: {'polygon': [[[1.93, 50.99], [2.09, 50.99], [2.09, 51.03], [1.93, 51.03], [1.93, 50.99]]], 'dates': ['1984-01-01', '2025-01-01'], 'sat_list': ['L5', 'L7', 'L8', 'L9'], 'sitename': 'HEMMESDEMARCK', 'filepath': '/home/athen

KeyboardInterrupt: 

QSocketNotifier: Invalid socket 87 and type 'Read', disabling...


# Time post processing

In [ ]:
import os
import json
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects

# Load the updated JSON file with inputs added
try:
    with open('json/sites_data_updated.json', 'r') as json_file:
        data_loaded = json.load(json_file)
        print("JSON data loaded successfully.")
except FileNotFoundError:
    print("The file 'sites_data_updated.json' does not exist.")
    data_loaded = {"sites": []}
except json.JSONDecodeError:
    print("Error decoding JSON from the file. Please check the file format.")
    data_loaded = {"sites": []}
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    data_loaded = {"sites": []}

# Process each site to define transects and generate time-series
for site in data_loaded["sites"]:
    try:
        sitename = site['sitename']
        inputs = site['inputs']

        # Debug: print inputs to trace errors
        print(f"Debug: Inputs for site {sitename}: {inputs}")

        # Load shoreline output file
        filepath = os.path.join(inputs['filepath'], sitename)
        try:
            with open(os.path.join(filepath, sitename + '_output.pkl'), 'rb') as f:
                output = pickle.load(f)
                print("Shoreline output loaded successfully.")
        except FileNotFoundError:
            print(f"Output file for site {sitename} not found. Skipping transect processing.")
            continue
        except pickle.PickleError:
            print(f"Error loading pickle file for site {sitename}.")
            continue
        except Exception as e:
            print(f"An unexpected error occurred during shoreline loading for site {sitename}: {e}")
            continue

        # Check if transects file exists
        transects_file = os.path.join(filepath, sitename + '_transects.pkl')
        if os.path.exists(transects_file):
            with open(transects_file, 'rb') as f:
                transects = pickle.load(f)
            print(f"Transects file already exists for site: {sitename}. Skipping transect drawing.")
        else:
            # Combine settings with inputs
            settings = {
                'cloud_thresh': 0.5,
                'dist_clouds': 300,
                'output_epsg': 28356,
                'check_detection': True,
                'adjust_detection': False,
                'save_figure': True,
                'min_beach_area': 1000,
                'min_length_sl': 500,
                'cloud_mask_issue': False,
                'sand_color': 'default',
                'pan_off': False,
                's2cloudless_prob': 60,
                'inputs': inputs,
            }

            # Draw shore-normal transects interactively
            print(f"Drawing transects interactively for site: {sitename}")
            %matplotlib qt
            transects = SDS_transects.draw_transects(output, settings)
            
            # Save transects to a file
            with open(transects_file, 'wb') as f:
                pickle.dump(transects, f)
            print(f"Transects saved for site: {sitename}")

        # Intersect shorelines with transects to obtain time-series of cross-shore distance
        settings_transects = {'along_dist': 25}
        cross_distance = SDS_transects.compute_intersection(output, transects, settings_transects)

        # Quality control intersections
        settings_transects_qc = {
            'along_dist': 25,
            'min_points': 3,
            'max_std': 15,
            'max_range': 30,
            'min_chainage': -100,
            'multiple_inter': 'auto',
            'auto_prc': 0.1,
        }
        cross_distance_qc = SDS_transects.compute_intersection_QC(output, transects, settings_transects_qc)

        # Despiking the time-series
        settings_outliers = {'max_cross_change': 40, 'otsu_threshold': [-.5, 0], 'plot_fig': True}
        cross_distance_qc = SDS_transects.reject_outliers(cross_distance_qc, output, settings_outliers)

        # Seasonal averaging and long-term trends
        season_colors = {'DJF':'C3', 'MAM':'C1', 'JJA':'C2', 'SON':'C0'}
        for key in cross_distance_qc.keys():
            chainage = cross_distance_qc[key]
            idx_nan = np.isnan(chainage)
            dates_nonan = [output['dates'][i] for i in np.where(~idx_nan)[0]]
            chainage = chainage[~idx_nan] 
            
            dict_seas, dates_seas, chainage_seas, list_seas = SDS_transects.seasonal_average(dates_nonan, chainage)
            trend, y = SDS_transects.calculate_trend(dates_seas, chainage_seas)
            
            fig, ax = plt.subplots(1, 1, figsize=[14, 4], tight_layout=True)
            ax.grid(which='major', linestyle=':', color='0.5')
            ax.set_title(f'Time-series at {key}', x=0, ha='left')
            ax.set(ylabel='distance [m]')
            ax.plot(dates_nonan, chainage, '+', lw=1, color='k', mfc='w', ms=4, alpha=0.5, label='raw datapoints')
            ax.plot(dates_seas, chainage_seas, '-', lw=1, color='k', mfc='w', ms=4, label='seasonally-averaged')
            for seas in dict_seas.keys():
                ax.plot(dict_seas[seas]['dates'], dict_seas[seas]['chainages'], 'o', mec='k', color=season_colors[seas], label=seas, ms=5)
            ax.plot(dates_seas, y, '--', color='b', label=f'trend {trend:.1f} m/year')
            ax.legend(loc='lower left', ncol=7, markerscale=1.5, frameon=True, edgecolor='k', columnspacing=1)
            fig.savefig(os.path.join(filepath, f'seasonal_averages_{key}.jpg'))

        # Monthly averages
        month_colors = plt.get_cmap('tab20')
        for key in cross_distance_qc.keys():
            chainage = cross_distance_qc[key]
            idx_nan = np.isnan(chainage)
            dates_nonan = [output['dates'][i] for i in np.where(~idx_nan)[0]]
            chainage = chainage[~idx_nan] 
            
            dict_month, dates_month, chainage_month, list_month = SDS_transects.monthly_average(dates_nonan, chainage)
            
            fig, ax = plt.subplots(1, 1, figsize=[14, 4], tight_layout=True)
            ax.grid(which='major', linestyle=':', color='0.5')
            ax.set_title(f'Time-series at {key}', x=0, ha='left')
            ax.set(ylabel='distance [m]')
            ax.plot(dates_nonan, chainage, '+', lw=1, color='k', mfc='w', ms=4, alpha=0.5, label='raw datapoints')
            ax.plot(dates_month, chainage_month, '-', lw=1, color='k', mfc='w', ms=4, label='monthly-averaged')
            for month in dict_month.keys():
                ax.plot(dict_month[month]['dates'], dict_month[month]['chainages'], 'o', mec='k', color=month_colors(month), label=month, ms=5)
            ax.legend(loc='lower left', ncol=7, markerscale=1.5, frameon=True, edgecolor='k', columnspacing=1)
            fig.savefig(os.path.join(filepath, f'monthly_averages_{key}.jpg'))

        # Save the time-series as CSV
        out_dict = dict([])
        out_dict['dates'] = output['dates']
        for key in transects.keys():
            out_dict[key] = cross_distance_qc[key]
        df = pd.DataFrame(out_dict)
        fn = os.path.join(filepath, 'transect_time_series.csv')
        df.to_csv(fn, sep=',')
        print(f"Time-series of the shoreline change along the transects saved as: {fn}")

    except KeyError as e:
        print(f"Key error: {e} for site: {site}")
    except Exception as e:
        print(f"An unexpected error occurred while processing transects for site: {sitename}. Error: {e}")

print("Transect processing completed.")
